# AI Planning

## Introduction

AI Planning is a field of Artificial Intelligence which explores the process of using autonomous techniques to solve planning and scheduling problems. A planning problem is one in which we have some initial starting state, which we wish to transform into a desired goal state through the application of a sequence of actions.

One branch of planning is State-Space Search in which we define a domain of possible states in our problem, and actions which result in transitioning between states, with the goal being to reach a predifined goal state(s).

## Best First Search

Best First Search is a family of search algorithms that are particularly useful for State-Space exploration.

The goal of Best First Search is to find a sequence of actions that will move us from the starting state to the goal state, while minimizing the cost of the journey which is defined by some cost function. Often the cost function will include a heuristic component which will allow us to estimate the cost of the path from the current state to a goal state.

### About heuristics
Generally in Best First Search algorithms that include a heuristic, the better it is at describing our distance from the goal state, the faster we will reach it, with "faster" being whatever fits the problem, whether it's the number of steps we had to take to reach our goal or some other cost function.

Two important terms that will help differentiate between Best First Search and its derivatives are Admissibility and Consistency.

A heuristic function is said to be admissible if its cost from any given state is never higher than the true cost required to reach a goal state from it, or in other words, it is always at least as "optimistic" as the real best solution.

Similarly, a heuristic function is said to be Consistent if for every state it holds true that the heuristic value of this state is greater or equal than the cost of moving to a neighboring state plus the heuristic value of that state. In other words, you can never "cheat" the heuristic valuation by just moving a little. As a side note, it can be proven that a consistent heuristic is also admissible.

In [1]:
import gym

taxi_env = gym.make("Taxi-v3").env

taxi_env.reset ()
taxi_env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [2]:
taxi_env.P

{0: {0: [(1.0, 100, -1, False)],
  1: [(1.0, 0, -1, False)],
  2: [(1.0, 20, -1, False)],
  3: [(1.0, 0, -1, False)],
  4: [(1.0, 16, -1, False)],
  5: [(1.0, 0, -10, False)]},
 1: {0: [(1.0, 101, -1, False)],
  1: [(1.0, 1, -1, False)],
  2: [(1.0, 21, -1, False)],
  3: [(1.0, 1, -1, False)],
  4: [(1.0, 17, -1, False)],
  5: [(1.0, 1, -10, False)]},
 2: {0: [(1.0, 102, -1, False)],
  1: [(1.0, 2, -1, False)],
  2: [(1.0, 22, -1, False)],
  3: [(1.0, 2, -1, False)],
  4: [(1.0, 18, -1, False)],
  5: [(1.0, 2, -10, False)]},
 3: {0: [(1.0, 103, -1, False)],
  1: [(1.0, 3, -1, False)],
  2: [(1.0, 23, -1, False)],
  3: [(1.0, 3, -1, False)],
  4: [(1.0, 19, -1, False)],
  5: [(1.0, 3, -10, False)]},
 4: {0: [(1.0, 104, -1, False)],
  1: [(1.0, 4, -1, False)],
  2: [(1.0, 24, -1, False)],
  3: [(1.0, 4, -1, False)],
  4: [(1.0, 4, -10, False)],
  5: [(1.0, 4, -10, False)]},
 5: {0: [(1.0, 105, -1, False)],
  1: [(1.0, 5, -1, False)],
  2: [(1.0, 25, -1, False)],
  3: [(1.0, 5, -1, False)

In [3]:
state = taxi_env.encode(1, 3, 4, 1) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

taxi_env.render()
taxi_env.unwrapped.s = state
taxi_env.render()

State: 177
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
import gym
from AI_agents.Environments.gym_problem import GymProblem
from AI_agents.Search.best_first_search import best_first_search, breadth_first_search, depth_first_search, a_star, depth_first_search_l
import AI_agents.Search.utils as utils
import AI_agents.Search.defs as defs
import AI_agents.Search.heuristic as heuristic


The following will create and recreate the taxi environment we want to explore

In [5]:
def create_env():

    # define the environment
    taxi_env = gym.make("Taxi-v3").env
    taxi_env.reset()
    init_state = taxi_env.encode(0, 3, 4, 1)  # (taxi row, taxi column, passenger index, destination index)
    taxi_row, taxi_col, pass_idx, dest_idx = taxi_env.decode(init_state)
    print(taxi_row)
    taxi_env.unwrapped.s = init_state
    print("State:", init_state)
    taxi_env.render()
    return taxi_env 


## Derivatives of Best First Search

Breadth First Search and Depth Fist Search can technically also be seen as special cases of Best First Search. In the former, the heuristic function assigns lower values the earlier a node was expanded, and in the latter the opposite is true: lower values are assigned to nodes that were discovered later. In practice this is done with LIFO and FIFO queues.

Let's run Breadth First Search:

In [6]:
taxi_env = create_env()

# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)


# perform BFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = breadth_first_search(problem=taxi_p,
                                                                                         log=True,
                                                                                         log_file=None,
                                                                                         iter_limit=defs.NA,
                                                                                         time_limit=defs.NA,
                                                                                        )


print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    taxi_p.env.render()


0
State: 77
+---------+
|R: | :_:G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 177>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 177>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 5 cur_node:<Node 277>
InMethod best_first_design(node): explored_count:5 cur_node:<Node 277>, node_eval_time:0.000
best_first_design(node) node number 6 cur_node:<Node 197>
InMethod best_first_design(node): explored_count:6 cur_node:<Node 197>, node_eval_time:0.000
bes

Let's see the same methods, but now with the explicit call to BestFirstSearch (which shows us how Breadth First Search is a special case of BestFirstSearch)

In [7]:
taxi_env = create_env()

# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform BFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = best_first_search(problem=taxi_p,
                                                                                      frontier=utils.FIFOQueue(),
                                                                                      closed_list=utils.ClosedListOfKeys(),
                                                                                      termination_criteria=utils.TerminationCriteriaGoalStateReached(),
                                                                                      evaluation_criteria=utils.EvaluationCriteriaGoalCondition(),
                                                                                      prune_func=None,
                                                                                      log=True, log_file=None,
                                                                                      iter_limit=defs.NA,
                                                                                      time_limit=defs.NA,
                                                                                      )
print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    taxi_p.env.render()

0
State: 77
+---------+
|R: | :_:G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 177>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 177>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 5 cur_node:<Node 277>
InMethod best_first_design(node): explored_count:5 cur_node:<Node 277>, node_eval_time:0.000
best_first_design(node) node number 6 cur_node:<Node 197>
InMethod best_first_design(node): explored_count:6 cur_node:<Node 197>, node_eval_time:0.000
bes

**And** now, let's run DFS

In [8]:
taxi_env = create_env()
# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform DFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = depth_first_search(problem=taxi_p,
                                                                                       log=True,
                                                                                       log_file=None,
                                                                                       iter_limit=defs.NA,
                                                                                       time_limit=defs.NA,
                                                                                       )

print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    taxi_p.env.render()

0
State: 77
+---------+
|R: | :_:G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 157>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 157>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 257>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 257>, node_eval_time:0.000
best_first_design(node) node number 5 cur_node:<Node 237>
InMethod best_first_design(node): explored_count:5 cur_node:<Node 237>, node_eval_time:0.000
best_first_design(node) node number 6 cur_node:<Node 217>
InMethod best_first_design(node): explored_count:6 cur_node:<Node 217>, node_eval_time:0.000
b

and now let's run DFS-L, which is a special form of DFS that includes a limit on the depth explored before backtracking to earlier nodes.

In [9]:
taxi_env = create_env()
# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform DFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = depth_first_search_l(problem=taxi_p,
                                                                                       max_depth=2,
                                                                                       log=True,
                                                                                       log_file=None,
                                                                                       iter_limit=defs.NA,
                                                                                       time_limit=defs.NA,
                                                                                       )

print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    taxi_p.env.render()

0
State: 77
+---------+
|R: | :_:G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 177>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 177>, node_eval_time:0.000
solution is: []
[]


### A* Algorithm

A* is a search algorithm derived from Best First Search, in which the cost function is $f = g + h$, where $g$ represents the cost of getting from the source to the current node, and h is the heuristic. The power of A* is in the fact that given an admissible heuristic it is guaranteed to return an optimal solution. This alone is not impressive, because BreadthFirstSearch also gives an optimal solution, the difference is that A* tends to do so while expanding a much smaller number of nodes, thus being more efficient.

![Astar](images/Astar.png)

Let's see how A-Star works on the same problem with a Manhatten Distance heuristic.

In [15]:
PASSENGER_IN_TAXI = 4  # passenger idx when in taxi

taxi_env = create_env()
locs = taxi_env.unwrapped.locs  # environment locations

def manhatten_dist(r1, c1, r2, c2):
    # calssic manhatten dist |row1 - row2| + |col1 - col2|
    return abs(r1 - r2) + abs(c1 - c2) 

def manhatten_heuristic(node):
    # decode state integer to interpretable values
    taxi_row, taxi_col, passenger_idx, dest_idx = taxi_env.decode(node.state.get_key())
    
    # split to 2 cases where the passenger is in the taxi and not in the taxi.
    if passenger_idx == PASSENGER_IN_TAXI:
        # dist from the taxi to the destination
        return manhatten_dist(taxi_row, taxi_col, *locs[dest_idx]) + 1  # include dropoff
    elif passenger_idx == dest_idx:
        # passenger has reached the destination. this is a goal state
        return 0
    else:
        # dist from the taxi to the passenger and from the passenger to the destination
        passenger_dist = manhatten_dist(taxi_row, taxi_col, *locs[passenger_idx])
        dest_dist = manhatten_dist(*locs[passenger_idx], *locs[dest_idx])
        return passenger_dist + dest_dist + 2  # include pickup and dropoff actions

0
State: 77
+---------+
|R: | :_:G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [16]:
# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform A*
[best_value, best_node, best_plan, explored_count, ex_terminated] = best_first_search(problem=taxi_p,
                                                                                      frontier=utils.PriorityQueue(manhatten_heuristic),
                                                                                      closed_list=utils.ClosedListOfKeys(),
                                                                                      termination_criteria=utils.TerminationCriteriaGoalStateReached(),
                                                                                      evaluation_criteria=utils.EvaluationCriteriaGoalCondition(),
                                                                                      prune_func=None,
                                                                                      log=True, 
                                                                                      log_file=None,
                                                                                      iter_limit=defs.NA,
                                                                                      time_limit=defs.NA,
                                                                                      )

print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    taxi_p.env.render()

best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 85>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 85>, node_eval_time:0.000
solution is: ['2', '5']
['2', '5']
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


## Discussion: Best First Search in a continuous space

Best First Search algorithms require a discrete state space, therefor a continous space requires adaptation to use these methods.

Representation challenges:
* Translation - a function from the continous space to the discrete space 
* Inversabilaty - the ability to get the original representation from the translation
* Granularity - "size" of each discrete "pixel"

